In [4]:
from torch_geometric.profile.utils import count_parameters
from myModels.GNNModel import GraphGNNModels
from myModels.OtherLayers import VOCNodeEncoder, GNNInductiveNodeHead

In [1]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
dataset = Planetoid(root="data/", name="Cora", transform=NormalizeFeatures())
in_channel, out_channel = dataset.num_features, dataset.num_classes

In [ ]:
from torch_geometric.nn.models import MLP
from myModels.ourModels import NodeIterativeGNN
him_dim=32
encoder = MLP(
            in_channels=in_channel,
            hidden_channels=him_dim,
            out_channels=him_dim,
            num_layers=1
            )
decoder = MLP(
                in_channels=him_dim,
                hidden_channels=him_dim,
                out_channels=out_channel,
                num_layers=1
            )
net = NodeIterativeGNN(layer_name="GCNConv",
                           hidden_size=him_dim,
                           train_schedule=[0.5, 0.5],
                           homogeneous_flag=False,
                           mlp_dropout=0,
                           gnn_dropout=0,
                           readout_name="Mean",
                           encoder=encoder,
                           decoder=decoder,
                           jk=None,
                           use_bn=False,
                           use_relu=True
                           )

In [2]:
from torch_geometric.datasets import LRGBDataset
dataset_root = "data/"
dataset_name = "PascalVOC-SP"
train_dataset = LRGBDataset(root=dataset_root, name=dataset_name, split="train")
val_dataset = LRGBDataset(root=dataset_root, name=dataset_name, split="val")
test_dataset = LRGBDataset(root=dataset_root, name=dataset_name, split="test")


In [6]:
from myModels.GNNModel import GraphGNNModels
from ogb.graphproppred.mol_encoder import AtomEncoder
from myModels.OtherLayers import GNNInductiveNodeHead
from torch_geometric.profile.utils import count_parameters
from myModels.ourModels import NodeIterativeGNN
hid_dim = 400
out_channel = train_dataset.num_classes
encoder = VOCNodeEncoder(hid_dim)
decoder = GNNInductiveNodeHead(in_dim=hid_dim, hid_dim=hid_dim, out_dim=out_channel, num_layers=3)
net = GraphGNNModels(architecture_name="DeepGNN",
                     layer_num=8,
                     layer_name="SAGEConv",
                     hidden_size=hid_dim,
                     readout_name='Mean',
                     encoder=encoder,
                     decoder=decoder
                     )
inet = NodeIterativeGNN(layer_name="GATConv",
                        hidden_size=hid_dim,
                        train_schedule=[0.6,0.6],
                        encoder=encoder,
                        decoder=decoder)
# print(str(net))
num_bn = count_parameters(net.gnn_module.batch_norms[0]) * len(net.gnn_module.batch_norms)
num_conf = count_parameters(net.confidence_module)
print(count_parameters(net) - num_bn - num_conf)
print(count_parameters(inet))


lala
aggr: max
2899221
498021


In [10]:
from myModels.ourModels import GraphIterativeGNN
hid_dim = 450
encoder = AtomEncoder(hid_dim)
decoder = GNNInductiveNodeHead(in_dim=hid_dim, hid_dim=hid_dim, out_dim=out_channel, num_layers=1)
iter_net = GraphIterativeGNN(layer_name="SAGEConv",
                             hidden_size=hid_dim,
                             train_schedule=[0.4, 0.5],
                             encoder=encoder,
                             decoder=decoder)
print(str(iter_net))
count_parameters(iter_net)

lala
aggr: max
GraphIterativeGNN(
  (embedding_module): AtomEncoder(
    (atom_embedding_list): ModuleList(
      (0): Embedding(119, 450)
      (1): Embedding(5, 450)
      (2-3): 2 x Embedding(12, 450)
      (4): Embedding(10, 450)
      (5-6): 2 x Embedding(6, 450)
      (7-8): 2 x Embedding(2, 450)
    )
  )
  (head_module): GNNInductiveNodeHead(
    (layer_post_mp): Sequential(
      (0): Linear(450, 21, bias=True)
    )
  )
  (readout_module): ReadoutLayers()
  (gnn_module): iterArch(
    (layer): GNNLayer(
      (layer): SAGEConv(
        (module): SAGEConv(450, 450, aggr=max)
      )
    )
    (readout): ReadoutLayers()
    (batch_norm): BatchNorm1d(450, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


494121

In [ ]:
from torch_geometric.nn import SAGEConv

sg = SAGEConv(10, 15, "max")